# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

geoapify_key

'82044decd9284a8a8703c7250bf0a219'

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,isafjordur,66.0755,-23.1240,-4.07,73,18,3.63,IS,1737586526
1,1,port-aux-francais,-49.3500,70.2167,5.10,78,68,15.74,TF,1737586527
2,2,edinburgh of the seven seas,-37.0676,-12.3116,17.12,85,82,1.32,SH,1737586528
3,3,princeville,45.7668,-61.2985,-10.38,79,100,5.66,CA,1737586529
4,4,bengkulu,-3.8004,102.2655,24.56,85,98,1.80,ID,1737586530


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    tiles = "OSM",
    geo = True,
    frame_width=800,
    size="Humidity",
    frame_height=600,
    color = "City",
    alpha=.7
)


# Display the map
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp= 15
wind_speed = 3
cloudiness=2

weatherdf = city_data_df[
    (city_data_df["Max Temp"]>= temp)&
    (city_data_df["Wind Speed"]<= wind_speed)&
    (city_data_df["Cloudiness"]<= cloudiness)
]

# Drop any rows with null values
weatherdf = weatherdf.dropna()

# Display sample data
weatherdf

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,alice springs,-23.7000,133.8833,32.75,15,1,1.03,AU,1737586420
31,31,remire-montjoly,4.9167,-52.2667,24.02,94,0,1.03,GF,1737586559
216,216,new kingman-butler,35.2650,-114.0323,15.43,7,1,2.24,US,1737586759
305,305,tomatlan,19.9333,-105.2500,29.94,27,0,2.78,MX,1737586856
399,399,clovis,36.8252,-119.7029,16.76,44,0,0.00,US,1737586962
416,416,pacific grove,36.6177,-121.9166,21.61,49,0,2.06,US,1737586981
498,498,tambacounda,13.7539,-13.7586,27.03,39,0,2.06,SN,1737587071
502,502,san blas,21.5167,-105.2667,21.97,59,0,1.83,MX,1737587075
508,508,coahuayana de hidalgo,18.7000,-103.6583,29.24,37,0,2.21,MX,1737587081
533,533,necochea,-38.5473,-58.7368,17.00,78,2,2.98,AR,1737587108


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weatherdf[["City", "Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,alice springs,AU,-23.7000,133.8833,15,
31,remire-montjoly,GF,4.9167,-52.2667,94,
216,new kingman-butler,US,35.2650,-114.0323,7,
305,tomatlan,MX,19.9333,-105.2500,27,
399,clovis,US,36.8252,-119.7029,44,
416,pacific grove,US,36.6177,-121.9166,49,
498,tambacounda,SN,13.7539,-13.7586,39,
502,san blas,MX,21.5167,-105.2667,59,
508,coahuayana de hidalgo,MX,18.7000,-103.6583,37,
533,necochea,AR,-38.5473,-58.7368,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories' : 'accommodation.hotel',
    'apiKey' : geoapify_key,
    'limit' : 30
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    
    longitude = row['Lng']
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] =  f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
alice springs - nearest hotel: Aurora Alice Springs
remire-montjoly - nearest hotel: Complexe Belova
new kingman-butler - nearest hotel: La Quinta Inn
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
clovis - nearest hotel: BEST WESTERN Clovis Cole
pacific grove - nearest hotel: Pacific Grove Inn
tambacounda - nearest hotel: Le relais de Tamba
san blas - nearest hotel: Suites San Blas
coahuayana de hidalgo - nearest hotel: No hotel found
necochea - nearest hotel: Cabañas Río Quequén
mukalla - nearest hotel: فندق الجابري


,City,Country,Lat,Lng,Humidity,Hotel Name
20,alice springs,AU,-23.7000,133.8833,15,Aurora Alice Springs
31,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
216,new kingman-butler,US,35.2650,-114.0323,7,La Quinta Inn
305,tomatlan,MX,19.9333,-105.2500,27,Hotel Hacienda Vieja Tomatlan
399,clovis,US,36.8252,-119.7029,44,BEST WESTERN Clovis Cole
416,pacific grove,US,36.6177,-121.9166,49,Pacific Grove Inn
498,tambacounda,SN,13.7539,-13.7586,39,Le relais de Tamba
502,san blas,MX,21.5167,-105.2667,59,Suites San Blas
508,coahuayana de hidalgo,MX,18.7000,-103.6583,37,No hotel found
533,necochea,AR,-38.5473,-58.7368,78,Cabañas Río Quequén


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    hover_cols=['City','Country','Humidity','Hotel Name'],
    color = "City"
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)